### SageMaker fine tune ChatGLM

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备requirements.txt
3. 准备s5cmd utility
4. 下载HF model，并上传到S3(optional)

In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

  Created wheel for sagemaker: filename=sagemaker-2.153.0-py2.py3-none-any.whl size=1008105 sha256=b5c59266f9d3f8b737aad3656323127b1d96ebc365d2213038e25c3985ac00cf
  Stored in directory: /home/ec2-user/.cache/pip/wheels/6a/0b/81/d7a0f0e734bc9f5ea74716acef6b397a6d473cb8e6b9b3f121
Successfully built sagemaker
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.0
    Uninstalling cloudpickle-2.2.0:
      Successfully uninstalled cloudpickle-2.2.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.146.0
    Uninstalling sagemaker-2.146.0:
      Successfully uninstalled sagemaker-2.146.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.11.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.2 which is incompatible.


In [3]:
#print('s3://{}/llm/models/'.format(sagemaker_session.default_bucket()))
#!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/
!curl -L https://github.com/peak/s5cmd/releases/download/v2.0.0/s5cmd_2.0.0_Linux-64bit.tar.gz | tar -xz && mv s5cmd ./ptuning/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4176k  100 4176k    0     0  5928k      0 --:--:-- --:--:-- --:--:-- 11.9M


In [4]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker-us-west-2-687912291502


In [5]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.1 MB/s eta 0:00:00


In [6]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "THUDM/chatglm-6b"#

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    #allow_patterns=allow_patterns,
)

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

In [7]:
print(model_download_path)
model_uri="s3://"+bucket+"/llm/model/chatglm/orignal/"
print(model_uri)

model/models--THUDM--chatglm-6b/snapshots/4d0fc39a58dcb747ab74ab2c4587bd66dcfa7e74
s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/


In [10]:
#using s5cmd util to fast upload model to s3
! ./ptuning/s5cmd sync ./model/models--THUDM--chatglm-6b/snapshots/4d0fc39a58dcb747ab74ab2c4587bd66dcfa7e74/ s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/
#!rm -rf model

cp model/models--THUDM--chatglm-6b/snapshots/4d0fc39a58dcb747ab74ab2c4587bd66dcfa7e74/tokenizer_config.json s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/tokenizer_config.json
cp model/models--THUDM--chatglm-6b/snapshots/4d0fc39a58dcb747ab74ab2c4587bd66dcfa7e74/quantization.py s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/quantization.py
cp model/models--THUDM--chatglm-6b/snapshots/4d0fc39a58dcb747ab74ab2c4587bd66dcfa7e74/configuration_chatglm.py s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/configuration_chatglm.py
cp model/models--THUDM--chatglm-6b/snapshots/4d0fc39a58dcb747ab74ab2c4587bd66dcfa7e74/README.md s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/README.md
cp model/models--THUDM--chatglm-6b/snapshots/4d0fc39a58dcb747ab74ab2c4587bd66dcfa7e74/LICENSE s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/LICENSE
cp model/models--THUDM--chatglm-6b/snapshots/4d0fc39a58dcb747ab74ab2c4587bd66dcfa7e74/py

### chatglm 官方P-tuning v2方式（单机单卡）
1:安装依赖lib   
2:准备数据集(本例以ADGEN 文本生成数据集为例，将解压后的 AdvertiseGen 目录放到本目录  
3:修改并bash运行 train.sh  

In [97]:
#!pip install rouge_chinese nltk jieba datasets
!git clone https://github.com/THUDM/ChatGLM-6B.git ./tmp/
#!pip install -r ChatGLM-6B/requirements.txt
!cp ChatGLM-6B/requirements.txt ChatGLM-6B/ptuning/

Cloning into './tmp'...
remote: Enumerating objects: 1094, done.
remote: Counting objects: 100% (530/530), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 1094 (delta 478), reused 473 (delta 457), pack-reused 564
Receiving objects: 100% (1094/1094), 7.91 MiB | 14.64 MiB/s, done.
Resolving deltas: 100% (647/647), done.


In [ ]:
#!cd ChatGLM-6B/ptuning/ && wget "https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1"
#!cd ChatGLM-6B/ptuning/ && mv "index.html?dl=1" dataset.tar.gz
#!cd ChatGLM-6B/ptuning/ && tar -xvf dataset.tar.gz
!./ChatGLM-6B/ptuning/s5cmd sync ChatGLM-6B/ptuning/AdvertiseGen/ s3://{bucket}/llm/chatglm/datasets/ 
#!rm -rf cd ChatGLM-6B/ptuning/dataset.tar.gz


In [ ]:
# define Training Job Name 
import time
from sagemaker.huggingface import HuggingFace
instance_type="ml.g4dn.2xlarge"

job_name = f'huggingface-chatglm-simple-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/simple/'.format(sagemaker_session.default_bucket())
output_dir = '/opt/ml/model/adgen-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'


instance_count = 1
#define the enviroment variables for your scripts.
environment = {
              'MODEL_S3_PATH'          : model_uri,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              #'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'TRAIN_STEPS'            : '50'
}

inputs={
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}


# create the Estimator
from sagemaker.pytorch import PyTorch

huggingface_estimator = HuggingFace(
    entry_point          = 'start_simple.py',          # user endpoint script
    source_dir           = './ptuning',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,           # Iam role used in training job to access AWS ressources, e.g. S3
    script_mode          = True,
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment
)

huggingface_estimator.fit(inputs=inputs)

In [ ]:
huggingface_estimator.model_data

In [ ]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/huggingface-chatglm-simple-2023-05-06-1-2023-05-06-14-24-12-728/output/model.tar.gz --recursive --human-readable --summarize

In [ ]:
#For local test only
!cd ./ptuning/&& bash train.sh

### chatglm 官方deepspeed方式（全参数的Finetune,单机多卡）
1: 准备deepspeed lib，并修改deepspeed.json    
2：数据集（以上一致）  
3：entrypoint start-single-node.py,设置num-gpus  
4：触发bash ds_train_finetune_single_node.sh 

In [13]:
instance_type = 'ml.g5.48xlarge'
if instance_type in [
    "ml.p3.16xlarge",
    "ml.p3dn.24xlarge",
    "ml.g5.48xlarge",
    "ml.p4d.24xlarge"    
]:
    processes_per_host = 8
elif instance_type == "ml.p2.16xlarge":
    processes_per_host = 16
elif instance_type == "ml.g4dn.2xlarge":
    processes_per_host = 1
else:
    processes_per_host = 4
    

print("processes_per_host is set to:", processes_per_host)

processes_per_host is set to: 8


In [ ]:
# define Training Job Name 
job_name = f'huggingface-chatglm-deepspeed-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/deepspeed-single-node/'.format(sagemaker_session.default_bucket())
output_dir = '/tmp/model/adgen-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'
#model_name_or_path = "/tmp/chatglm/"


instance_count = 1
#define the enviroment variables for your scripts.
environment = {
              
              'MODEL_S3_PATH'          : model_uri,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              #'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'NUM_GPUS'               : str(processes_per_host),
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'TRAIN_STEPS'            :'20'
}

inputs={
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}


# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'start-single-node.py',          # user endpoint script
    source_dir           = './ptuning',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    script_mode          = True,
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment,
)
huggingface_estimator.fit(inputs=inputs)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-chatglm-deepspeed-2023-05-1-2023-05-11-09-58-06-102


Using provided s3_resource
2023-05-11 09:58:08 Starting - Starting the training job.

In [ ]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/deepspeed-single-node/adgen-chatglm-6b-ft/

### chatglm deepspeed 多机多卡改造
1: 准备deepspeed lib，并修改deepspeed.json    
2：数据集（以上一致）  
3：entrypoint start.py,设置torch distribute launch configure  
4：触发bash ds_train_finetune.sh 

In [ ]:
# define Training Job Name 
job_name = f'huggingface-chatglm-deepspeed-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/deepspeed-mutip-nodes/'.format(sagemaker_session.default_bucket())
output_dir = '/tmp/model/adgen-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'


instance_count = 2
#define the enviroment variables for your scripts.
environment = {
              'NODE_NUMBER'            : str(instance_count),
              'MODEL_S3_PATH'          : model_uri,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              #'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'NUM_GPUS'               : str(processes_per_host),
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'TRAIN_STEPS'            :'50'
}

inputs={
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}


# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'start.py',          # user endpoint script
    source_dir           = './ptuning',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    script_mode          = True,
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment,
)
huggingface_estimator.fit(inputs=inputs)

In [ ]:
#!aws s3 rm s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/deepspeed/adgen-chatglm-6b-ft/checkpoint-50 --recursive
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/deepspeed-mutip-nodes/adgen-chatglm-6b-ft/

In [ ]:
##for local test only
!cd ./ChatGLM-6B/ptuning/ && bash ./ds_train_finetune.sh

In [153]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/

                           PRE chatglm/
                           PRE model/
                           PRE models/
